In [1]:
import numpy as np
import pandas as pd
import psycopg2

##### Loading data

In [2]:
transactions = pd.read_csv(r"transactional_data.csv")
product_md = pd.read_csv(r"product_master_data.csv")
customer_md = pd.read_csv(r"customer_master_data.csv")

In [6]:
display(transactions.head())
display(product_md.head())
display(customer_md.head())

,Unnamed: 0,orderID,Customer_ID,Product_ID,quantity,date
0,0,9914432,1000001,P00069042,1,2020-12-29
1,1,1676537,1000001,P00248942,3,2017-03-27
2,2,8910457,1000001,P00087842,1,2017-08-20
3,3,5044982,1000001,P00085442,1,2019-03-03
4,4,4176351,1000002,P00285442,1,2018-07-11


,Unnamed: 0,Product_ID,Product_Category,price$,storeID,supplierID,storeName,supplierName
0,0,P00069042,Home & Kitchen,77.51,2,13,Tech Haven,Samsung Electronics
1,1,P00248942,Grocery,9.63,6,9,Photo World,Canon Inc.
2,2,P00087842,Pets,31.66,4,18,Game Zone,Razer Inc.
3,3,P00085442,Pets,23.32,1,16,Electro Mart,Sony Corporation
4,4,P00285442,Toys,34.71,3,39,Sound Zone,Sonos Inc.


,Unnamed: 0,Customer_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status
0,0,1000001,F,0-17,10,A,2,0
1,4,1000002,M,55+,16,C,4,0
2,5,1000003,M,26-35,15,A,3,0
3,6,1000004,M,46-50,7,B,2,1
4,9,1000005,M,26-35,20,A,1,1


In [10]:
print(transactions.dtypes, "\n")
print(product_md.dtypes, "\n")
print(customer_md.dtypes, "\n")

Unnamed: 0      int64
orderID         int64
Customer_ID     int64
Product_ID     object
quantity        int64
date           object
dtype: object 

Unnamed: 0            int64
Product_ID           object
Product_Category     object
price$              float64
storeID               int64
supplierID            int64
storeName            object
supplierName         object
dtype: object 

Unnamed: 0                     int64
Customer_ID                    int64
Gender                        object
Age                           object
Occupation                     int64
City_Category                 object
Stay_In_Current_City_Years     int64
Marital_Status                 int64
dtype: object 



##### Connecting to postgresql database

In [ ]:
conn = psycopg2.connect(
    host="host",
    database="walmartDW",
    user="user",
    password="password",
    port="port"
)

cur = conn.cursor()

In [85]:
# sample query to check if connection is active
cur.execute("SELECT version();")
print(cur.fetchall())

[('PostgreSQL 17.6 on x86_64-windows, compiled by msvc-19.44.35217, 64-bit',)]


#### Inserting master data from tables into the database 

In [86]:
supplier_df = product_md[['supplierID', 'supplierName']].drop_duplicates()
supplier_df

,supplierID,supplierName
0,13,Samsung Electronics
1,9,Canon Inc.
2,18,Razer Inc.
3,16,Sony Corporation
4,39,Sonos Inc.
17,17,Garmin Ltd.
39,51,Pakistan


In [88]:
store_df = product_md[['storeID', 'storeName']].drop_duplicates()
store_df

,storeID,storeName
0,2,Tech Haven
1,6,Photo World
2,4,Game Zone
3,1,Electro Mart
4,3,Sound Zone
10,5,InnoTech
17,7,Health Zone
39,51,Pakistan


In [89]:
product_df = product_md[['Product_ID', 'Product_Category', 'price$']].drop_duplicates()
product_df

,Product_ID,Product_Category,price$
0,P00069042,Home & Kitchen,77.51
1,P00248942,Grocery,9.63
2,P00087842,Pets,31.66
3,P00085442,Pets,23.32
4,P00285442,Toys,34.71
...,...,...,...
3626,P00375436,Shoes,69.03
3627,P00372445,Shoes,57.86
3628,P00370293,Jewelry & Accessories,10.38
3629,P00371644,Shoes,63.21


In [ ]:
# load supplier data into supplier
supplier_query = """
    INSERT INTO demoDB.supplier
    (supplier_id, supplierName)
    VALUES(%s, %s);
"""

# load store data
store_query = """
    INSERT INTO demoDB.store
    (store_id, storeName)
    VALUES(%s, %s);
"""

# load product data
product_query = """
    INSERT INTO demoDB.product
    (product_id, product_category, price)
    VALUES(%s, %s, %s);
"""

values = list((supplier_df[["supplierID", "supplierName"]]).itertuples(index=False, name=None))
cur.executemany(supplier_query, values)
conn.commit()

values = list((store_df[["storeID", "storeName"]]).itertuples(index=False, name=None))
cur.executemany(store_query, values)
conn.commit()

values = list((product_df[['Product_ID', 'Product_Category', 'price$']]).itertuples(index=False, name=None))
cur.executemany(product_query, values)
conn.commit()

In [ ]:
customer_query = """
    INSERT INTO demoDB.customer
    (customer_id, gender, age_group, occupation, city_category, marital_status, stay_in_current_city_years)
    VALUES(%s, %s, %s, %s, %s, %s, %s);
"""

values = list((customer_md[["customer_id", "gender", "age", "occupation", "city_category", "marital_status", "stay_in_current_city_years"]]).itertuples(index=False, name=None))
cur.executemany(customer_query, values)
conn.commit()


In [ ]:
cur.close()
conn.close()
print("Connection closed. \n")